In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import pymorphy2

In [2]:
test_data = pd.read_csv('test_data.csv')

In [3]:
def parsing(data):
    
    # оставляем реплики исследуемой роли, удаляем неинформативные столбцы
    data=data[data['role']=='client'].drop(['role','line_n'], axis =1)
    
    # создаем пустой датасет с номерами диалогов для дальнейшего заполнения
    dlg_list = data.dlg_id.unique()
    result = pd.DataFrame(dlg_list).rename(columns = {0 :'dlg_id'})
    
    # функция для извлечения реплики с приветствием по списку ключевых слов
    def greeting(text):
        greeting_list = ['доброе утро', 'добрый день', 'добрый вечер', 'здравствуйте', 'привет', 'приветствую']
        for greet in greeting_list:
            if greet in text:
                return text
                break
    
    data['greeting'] = data['text'].apply(greeting)
    
    # функция для извлечения имени менеджера по списку предшествующих вводных слов. Поиск имени в строке проводится с помощью морфологичесого анализа pymorphy2
    def get_name(text):
        intro_list = ['меня зовут', 'это','меня',' я ', 'я ']
        name = None
        
        def find_name(sentence):
            morph = pymorphy2.MorphAnalyzer()
            for word in nltk.word_tokenize(sentence):
                for p in morph.parse(word):
                    if 'Name' in p.tag and p.score >= 0.4:
                        return p.normal_form

        for i in intro_list:
            if i in text.lower():
                name = find_name(text.lower())
            else:
                continue
            break        
                        
        if name is not None:
            if i == 'меня':
                intro = ' '+i+' '+name+' '+'зовут'
                if intro in text.lower():
                    return name

            elif i == 'я ':
                if text.lower()[0] == 'я':
                    intro = i+' '+name
                    if intro in text.lower():
                        return name
             
            else:
                intro = i+' '+name
                if intro in text.lower():
                    return name

    data['name'] = data['text'].apply(get_name)
    
    # функция для извлечения реплики, где менеджер представился (по условию наличия имени менеджера)
    def introduction (col_1, col_2):
        if col_1 is not None:
            return col_2
     
    data['introduction'] = data.apply(lambda col: introduction(col['name'], col['text']), axis=1)
          
    # функция для извлечения названия компании по вводному слову. Анализ названия проводится с помощью морфологичесого анализа pymorphy2
    def get_org_name(text):
        text = re.sub('[^А-Яа-яЁё]', ' ', text) 
        text = text.lower()
        text = text.split() 
        trigger = 'компани'
        for i in range(0, len(text)):
            if trigger in text[i]:
                morph = pymorphy2.MorphAnalyzer()
                parsed = morph.parse(text[i+1])
                for variant in parsed:
                    if {'NOUN'} in variant.tag and variant.score >= 0.03:
                        return text[i+1]
    
    data['org_name'] = data['text'].apply(get_org_name)
    
    # функция для извлечения реплики с прощанием по списку ключевых слов
    def farewell(text):
        farewell_list = ['до свидания', 'всего хорошего', 'всего доброго', 'до скорого', 'прощайте']
        for greet in farewell_list:
            if greet in text:
                return text
                break
                
    data['farewell'] = data['text'].apply(farewell)
    
    # добавляем результаты парсинга по диалогам в итоговый датасет
    feature_list = ['greeting','name','introduction','org_name','farewell']
    for feature in feature_list :
        result = result.merge(data.drop(['text'], axis =1)[['dlg_id', feature]][data[feature].notnull()], on=['dlg_id'], how='left')
    
    # функция для проверки требования к менеджеру «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом».
    def requirement(col_1, col_2):
        if col_1 is np.nan or col_2 is np.nan:
            return False
        else:
            return True
      
    # добавляем выполнение требования к менеджеру True/False в итоговый датасет
    result['man_requirement'] = result.apply(lambda col: requirement(col['greeting'], col['farewell']), axis=1)
    
    return result

In [4]:
%%time
# результат парсинга на тестовом наборе данных
parsing(test_data)

CPU times: user 8.2 s, sys: 979 ms, total: 9.18 s
Wall time: 9.18 s


,dlg_id,greeting,name,introduction,org_name,farewell,man_requirement
0,0,Алло здравствуйте,ангелина,Меня зовут ангелина компания диджитал бизнес з...,диджитал,Всего хорошего до свидания,True
1,1,Алло здравствуйте,ангелина,Меня зовут ангелина компания диджитал бизнес з...,диджитал,Угу да вижу я эту почту хорошо тогда исправлю ...,True
2,2,Алло здравствуйте,ангелина,Меня зовут ангелина компания диджитал бизнес з...,диджитал,NaN,False
3,3,Алло дмитрий добрый день,максим,Добрый меня максим зовут компания китобизнес у...,китобизнес,Угу все хорошо да понедельника тогда всего доб...,True
4,4,NaN,NaN,NaN,NaN,Во вторник все ну с вами да тогда до вторника ...,False
5,5,NaN,анастасия,Да это анастасия,NaN,Ну до свидания хорошего вечера,False
